In [ ]:
import heapq

class Node:
    def __init__(self, position, g, h):
        self.position = position  # (x, y) coordinates
        self.g = g                # Cost from start to this node
        self.h = h                # Heuristic cost to the goal
        self.f = g + h            # Total cost (g + h)

    def __lt__(self, other):
        return self.f < other.f

def heuristic(a, b):
    """Calculate the Manhattan Distance heuristic."""
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def a_star_search(grid):
    start = (0, 0)  # Starting point
    goal = (4, 4)   # Goal point
    open_list = []  # Nodes to be evaluated
    closed_set = set()  # Nodes already evaluated

    # Initialize the start node
    start_node = Node(start, 0, heuristic(start, goal))
    heapq.heappush(open_list, start_node)

    # To reconstruct the path
    came_from = {}

    while open_list:
        # Get the node with the lowest f cost
        current_node = heapq.heappop(open_list)

        # Check if we reached the goal
        if current_node.position == goal:
            path = []
            while current_node.position in came_from:
                path.append(current_node.position)
                current_node = came_from[current_node.position]
            path.append(start)
            path.reverse()  # Reverse to get the path from start to goal
            return path

        closed_set.add(current_node.position)

        # Explore neighbors (4 possible directions)
        for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            neighbor = (current_node.position[0] + dx, current_node.position[1] + dy)

            # Check if the neighbor is within bounds and is not an obstacle
            if (0 <= neighbor[0] < 5 and 0 <= neighbor[1] < 5 and
                grid[neighbor[0]][neighbor[1]] == 0 and
                neighbor not in closed_set):

                g_score = current_node.g + 1  # Cost to reach this neighbor
                h_score = heuristic(neighbor, goal)  # Heuristic for neighbor
                neighbor_node = Node(neighbor, g_score, h_score)

                # If the neighbor is not already in the open list
                if neighbor not in [n.position for n in open_list]:
                    came_from[neighbor] = current_node
                    heapq.heappush(open_list, neighbor_node)

    return None  # No path found

def mark_path(grid, path):
    """Mark the path on the grid."""
    for x, y in path:
        grid[x][y] = 'P'
    return grid

# Define the grid with obstacles (1 = obstacle, 0 = free space)
grid = [
    [0, 0, 0, 1, 0],
    [1, 1, 0, 1, 0],
    [0, 0, 0, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0]
]

# Execute A* search
path = a_star_search(grid)
if path:
    marked_grid = mark_path(grid, path)
    for row in marked_grid:
        print(row)
else:
    print("No path found.")


['P', 'P', 'P', 1, 0]
[1, 1, 'P', 1, 0]
[0, 0, 'P', 'P', 'P']
[0, 1, 1, 1, 'P']
[0, 0, 0, 0, 'P']


In [ ]:
import random

def create_board(queens):
    """Creates an 8x8 board with queens placed at given rows in random columns."""
    board = [-1] * 8  # Initialize a board with -1 (no queen)
    for i in range(8):
        board[i] = random.randint(0, 7)  # Random column for each row
    return board

def calculate_conflicts(board):
    """Calculate the number of conflicts for the current board configuration."""
    conflicts = 0
    for i in range(8):
        for j in range(i + 1, 8):
            # Check if two queens are in the same column or diagonal
            if board[i] == board[j] or abs(board[i] - board[j]) == abs(i - j):
                conflicts += 1
    return conflicts

def get_neighbors(board):
    """Generate all neighboring states by moving one queen."""
    neighbors = []
    for row in range(8):
        for col in range(8):
            if col != board[row]:  # Move queen to a different column
                new_board = board[:]
                new_board[row] = col  # Change the column of the queen
                neighbors.append(new_board)
    return neighbors

def hill_climbing():
    """Main function to perform the Hill-Climbing algorithm for the 8-Queens problem."""
    current_board = create_board(8)  # Start with a random board
    current_conflicts = calculate_conflicts(current_board)

    while current_conflicts > 0:
        neighbors = get_neighbors(current_board)
        next_board = None
        next_conflicts = float('inf')

        # Find the neighbor with the least number of conflicts
        for neighbor in neighbors:
            conflict_count = calculate_conflicts(neighbor)
            if conflict_count < next_conflicts:
                next_conflicts = conflict_count
                next_board = neighbor

        # If no better neighbor is found, we are stuck
        if next_conflicts >= current_conflicts:
            print("No solution found.")
            return None

        current_board = next_board
        current_conflicts = next_conflicts

    return current_board

# Execute the Hill-Climbing algorithm
solution = hill_climbing()
if solution:
    print("Solution found:")
    for row in range(8):
        line = ['.'] * 8  # Create an empty row
        line[solution[row]] = 'Q'  # Place a queen in the appropriate column
        print(' '.join(line))
else:
    print("No solution.")


No solution found.
No solution.


In [ ]:
import math

def print_board(board):
    """Print the current state of the board."""
    for row in board:
        print(" | ".join(row))
        print("-" * 9)

def is_winner(board, player):
    """Check if the specified player has won the game."""
    # Check rows, columns, and diagonals for a win
    return any(all(cell == player for cell in row) for row in board) or \
           any(all(board[i][j] == player for i in range(3)) for j in range(3)) or \
           all(board[i][i] == player for i in range(3)) or \
           all(board[i][2 - i] == player for i in range(3))

def get_available_moves(board):
    """Return a list of available moves (empty cells)."""
    return [(r, c) for r in range(3) for c in range(3) if board[r][c] == " "]

def minimax(board, depth, is_maximizing):
    """Implement the Mini-Max algorithm."""
    if is_winner(board, 'O'):  # AI is 'O'
        return 1  # AI wins
    if is_winner(board, 'X'):  # Player is 'X'
        return -1  # Player wins
    if not get_available_moves(board):
        return 0  # Tie

    if is_maximizing:  # AI's turn
        best_score = -math.inf
        for r, c in get_available_moves(board):
            board[r][c] = 'O'  # Make the move
            score = minimax(board, depth + 1, False)  # Call minimax recursively
            board[r][c] = " "  # Undo the move
            best_score = max(score, best_score)  # Update best score
        return best_score
    else:  # Player's turn
        best_score = math.inf
        for r, c in get_available_moves(board):
            board[r][c] = 'X'  # Make the move
            score = minimax(board, depth + 1, True)  # Call minimax recursively
            board[r][c] = " "  # Undo the move
            best_score = min(score, best_score)  # Update best score
        return best_score

def best_move(board):
    """Determine the best move for the AI using Mini-Max."""
    best_score = -math.inf
    move = None
    for r, c in get_available_moves(board):
        board[r][c] = 'O'  # Make the move
        score = minimax(board, 0, False)  # Evaluate the move
        board[r][c] = " "  # Undo the move
        if score > best_score:
            best_score = score  # Update best score
            move = (r, c)  # Update best move
    return move

# Initialize the game board
board = [[" " for _ in range(3)] for _ in range(3)]
while True:
    print_board(board)

    if not get_available_moves(board):
        print("It's a tie!")
        break

    # Player X's turn
    while True:
        try:
            row = int(input("Enter your move row (0, 1, or 2): "))
            col = int(input("Enter your move column (0, 1, or 2): "))
            if (row, col) in get_available_moves(board):
                board[row][col] = 'X'  # Place player's move
                break
            else:
                print("Invalid move. Try again.")
        except (ValueError, IndexError):
            print("Invalid input. Please enter numbers 0, 1, or 2.")

    if is_winner(board, 'X'):
        print_board(board)
        print("X wins!")
        break

    # AI's turn (O)
    move = best_move(board)
    if move:
        board[move[0]][move[1]] = 'O'  # Place AI's move
        if is_winner(board, 'O'):
            print_board(board)
            print("O wins!")
            break


  |   |  
---------
  |   |  
---------
  |   |  
---------
